# Disclaimer 

Спочатку трошки про мене: я джун у фронтенді дев відділу ФІОТ. <br> 
Мій досвід не великий - щось довкола 10 тасків, де суттєвих, що потребують досвіду ( feature ) меншість. <br> 
Всі твердження виходять лише з мого досвіду. 

# Speeding up work at the cost of stupidity.  
Я лінивий. 


<br> Що ж. Для кінцевого результату нам важливо декілька речей
 - Те, як правильно модель буде розуміти поставлені задачі (bufix, enhancement, feature, etc.) 
 - Планування - як добре модель може розписати покрокове рішення задачі 
 - Написання коду - наскільки він валідний 

<br> Щоб досягти цього  існують декілька варіантів:
 1. запхати весь код проєкту у модель та сподіватися на краще 
 2. розібрати декілька іш'юсів з покроковим поясненням кожної дії 
або ж обидва пункти одразу 

<br> Що важливо? 
1. Якщо модель не знатиме вже написаного коду, вона не зможе давати точні відповіді та вказувати на конкретні помилки / виправлення. Таким чином її функціонал обмежується до абстрактних порад на основі вже вивченого з інтернету коду. 
Це проблема, яку я намагаюсь вирішити, оскільки такий ефект рівносильний тому, щоб запхати код у чатгпт без будь-яких додаткових дій. 

2. У більшості випадків, модель так чи інакше не здатна вигадати геніальне рішення для нової фічі. ( хоча шаблонні рішення повинні даватись легко )
З цим нічого не зробиш, оскільки у людського розробника досвіду куди більше, та людина врешті-решт може креативно мислити та комунікувати.

Перед тим, як переходити до створення датасету, давайте проведемо невеличке дослідження, все ж, ми тут датасаєнсом займаємось! <br>
З [треду](https://www.reddit.com/r/ChatGPT/comments/11ax0cy/is_it_possible_to_give_chatgpt_whole_of_my/) на реддіті, який чудово описує перше питання яке постало перед нами. 
<br>У мене вийшло дістати трошки корисної інформації, а саме те, що велика кількість коду не вміститься у короткостроковій пам'яті гпт, для цього існуює embedingsAPI
<br>А також та знайшов [cookbook](https://cookbook.openai.com/examples/get_embeddings_from_dataset) яким демонструє як отримати embeddings з датасету 

Що ж таке ці embeddings, та з чого вони зліплені? [opeanAI](https://platform.openai.com/docs/guides/embeddings#:~:text=OpenAI's%20text%20embeddings%20measure%20the,related%20text%20strings%20are%20recommended)
<br> Одразу відповідаючи на це питання: це векторна репрезентація слів, де вектор не тільки позначає саме по собі слово, а також його важливість та як воно відноситься до інших слів. 
<br> Те, як слова відносяться одне до одного визначити просто: це відстань між їх векторами.

А тепер давайте спробуємо! 
<br> Для наглядності та економії часу у прикладі я буду використовувати реальний код проєкту.

Спочатку імпортуємо всі необхідні бібліотеки 

In [1]:

import os
import sys
import subprocess
import shutil
import pandas as pd
import tiktoken
#from utils.embeddings_utils import get_embedding
from dotenv import load_dotenv
from openai import OpenAI
import openai
load_dotenv('../../.env')

True

Визначемо шлях завантаження, посилання на сам репозиторій та файл який ми будемо використовувати для прикладу 
<br> Зокрема зазначимо модель, яку будемо використовувати 

In [2]:
URL = "https://github.com/fictadvisor/fictadvisor-web.git" 
PATH = '../../../assets/repositories/frontend/'
FILE = 'src/components/pages/personal-teacher-page/PersonalTeacherPage.tsx'
MODEL =  "text-embedding-ada-002"
GPTMODEL = "gpt-3.5-turbo"

In [3]:
to_clone = False # Щоб не клонувати кожен раз репозиторій, коли він вже є, 
if to_clone:
    directory = '../assets/repositories/frontend/'
    if os.path.exists(directory):
        shutil.rmtree(directory)
    subprocess.run(["git", "clone", URL, PATH])

Прочитаємо наш файл

<br> Оскільки ми працюємо з трансформером, кожне слово повинно бути перетворене на токен ( у нашому випадку, у вигляді вектору ), оскільки мережі не розуміють слів напряму. 
<br> Існує багато способів токенізації, але зараз ми просто використаємо той, що пропонують самі openAI. 
<br> Чому це для нас взагалі важливо? 
<br> У кожної з запропонованих моделей на сьогоднішній день існує максимальна кількість токенів, що є для нас проблемою.
<br> Але не тільки у цьому справа: за велику кількість токенів доведеться і більше платити :>

<br> Годі слів, час пробувати!

Для початку відкриємо не великий за змістом файл, скоріше серендій з оглядкою на мій досвід.

In [4]:
with open(PATH + FILE, 'r') as file:
    filedata = file.read()
print(filedata)

import {
  createContext,
  Dispatch,
  FC,
  SetStateAction,
  useEffect,
  useState,
} from 'react';
import { useRouter } from 'next/router';

import Breadcrumbs from '@/components/common/ui/breadcrumbs';
import PersonalTeacherCard from '@/components/common/ui/cards/personal-teacher-card';
import Progress from '@/components/common/ui/progress';
import PersonalTeacherTabs from '@/components/pages/personal-teacher-page/personal-teacher-tabs';
import styles from '@/components/pages/personal-teacher-page/PersonalTeacherPage.module.scss';
import {
  PersonalTeacherPageProps,
  TeachersPageTabs,
} from '@/components/pages/personal-teacher-page/utils';
import useTabState from '@/hooks/use-tab-state';
import useToast from '@/hooks/use-toast';
import { Teacher } from '@/types/teacher';

// TODO: move context to separate folder, move types to separate folder
export interface TeacherContext {
  floatingCardShowed: boolean;
  setFloatingCardShowed: Dispatch<SetStateAction<boolean>>;
  teacher: T

Та порахуємо скільки буде коштувати запхати його у нашу модель! ( багато ) ( бюджет СР не резиновий )
<br> Для початку завантажимо токенайзер для нашої моделі

In [5]:
client = OpenAI()
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
# Downloading encodings for our model 
encoding = tiktoken.encoding_for_model(MODEL)

In [6]:
def count_tokens(text):
    return len(encoding.encode(text))

count_tokens(filedata)

744

744 Токени це велике число, не підйомне для мого гаманця, враховуючи що на проєкті fictadvisor більш ніж 950 файлів! 
<br> Давайде дізнаємось його ціну, можливо все не так погано

In [7]:
def calculate_price(tokens):
    """ 
    Model            Training                Input usage             Output usage
    gpt-3.5-turbo    $0.0080 / 1K tokens	 $0.0030 / 1K tokens     $0.0060 / 1K tokens
    """
    return tokens/1000*0.0060

print(f"{count_tokens(filedata)} tokens for {calculate_price(count_tokens(filedata))}$")

744 tokens for 0.004464$


Отже, багатенько, як на один файл. Пропоную порахувати скільки коштувало б скормити весь код fictadvisor frontend нашій моделі! 

In [8]:
def tokens_in_each_file(directory):
    total_tokens = 0
    num_files = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                filedata = file.read()
            tokens = count_tokens(filedata)
            total_tokens += tokens
            num_files += 1
    return  {'total_tokens': total_tokens, 'total_files': num_files} 


data = tokens_in_each_file(PATH) 
print(
    f"Total tokens: {data['total_tokens']}" 
    + f"\n Total amount of files: {data['total_files']}" 
    + f"\n Average tokens per file: {data['total_tokens'] / data['total_files'] }"
    + f"\n Cost for all files: {calculate_price(data['total_tokens'])}$"
    + f"\n Cost per file: {calculate_price(data['total_tokens']) / data['total_files']}$"
)

Total tokens: 36302884
 Total amount of files: 1024
 Average tokens per file: 35452.03515625
 Cost for all files: 217.817304$
 Cost per file: 0.2127122109375$


217 Доларів, що ж, я не можу собі дозволити таке задоволення. Треба трошки подумати!
<br> Перш за все, варто зауважити, що наведений код бере абсолютно всі файли які є у проєкті, це можуть бути і картинки також, а картинки можуть займати дуже багато місця
<br> Окрім  того, код не прибирає імпорти, які є у кожному файлі, і їх на справді дуже багато. Для тренування це не така важлива інформація, а ось грошей тягне багато. 
<br> Далі: пробіли, код не розділяється на слова, тому зайві пробіли можуть бути токенізовані, цього теж слід уникати 
<br> Тому давайте перерахуємо вартість!

In [9]:
def calculate_average_tokens_per_file(directory):
    total_tokens = 0
    num_files = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            # skip non-js/ts files
            if not (filename.endswith('.js') or filename.endswith('.jsx') or filename.endswith('.ts') or filename.endswith('.tsx')):
                continue
            
            file_path = os.path.join(dirpath, filename)
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                filedata = file.readlines()
            # skip imports and delete unnecessary spaces
            
            filedata = ''.join([line for line in filedata if 'import' not in line])
            filedata = ' '.join(word.strip() for line in filedata.split() for word in line.split())
            
            tokens = count_tokens(filedata)
            total_tokens += tokens
            num_files += 1
    return {'total_tokens': total_tokens, 'total_files': num_files}

data = calculate_average_tokens_per_file(PATH) 
print(
    f"Total tokens: {data['total_tokens']}" 
    + f"\n Total amount of files: {data['total_files']}" 
    + f"\n Average tokens per file: {data['total_tokens'] / data['total_files'] }"
    + f"\n Cost for all files: {calculate_price(data['total_tokens'])}$"
    + f"\n Cost per file: {calculate_price(data['total_tokens']) / data['total_files']}$"
)

Total tokens: 332705
 Total amount of files: 897
 Average tokens per file: 370.9085841694537
 Cost for all files: 1.99623$
 Cost per file: 0.0022254515050167223$


Що ж, добре, той факт, що ми не розоримося на цьому проєкті не може не радувати.  
<br> Давайте все-ж таки спробуємо зробити з цього embedding

Для порівняння, попросимо зробити це завдання нашу модель до її файнтюнінгу

In [10]:
query = "how do i do PersonalTeacherPage using React, Next, MUI for my website?"
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": ""},
    {"role": "user", "content": f"{query}"}
  ]
)

In [11]:
print(completion.choices[0].message.content)  

To create a PersonalTeacherPage using React, Next.js, and Material-UI (MUI) for your website, you can follow these steps:

Step 1: Set up your development environment
- Make sure you have Node.js and npm installed on your machine.

Step 2: Create a new Next.js project
- Open your terminal and run the following command to create a new Next.js project: 
```
npx create-next-app personal-teacher-page
```

Step 3: Install Material-UI
- Navigate to your project directory:
```
cd personal-teacher-page
```
- Run the following command to install Material-UI:
```
npm install @material-ui/core
```

Step 4: Create the PersonalTeacherPage component
- Inside the "pages" directory of your project, create a new file called "PersonalTeacherPage.js".
- Import React and Material-UI components in the file:
```javascript
import React from 'react';
import { makeStyles } from '@material-ui/core/styles';
import Container from '@material-ui/core/Container';
import Typography from '@material-ui/core/Typography'

Щож, очікувано, що він не знає відповіді на це питання
<br> Саме час це виправити!

In [12]:
import json

optimized = filedata
optimized = ''.join([line for line in optimized.split("\n") if 'import' not in line])
optimized = ' '.join(word.strip() for line in optimized.split() for word in line.split())

sample = {"messages": 
[
    {"role": "system", "content": ""}, 
    {"role": "user", "content": 'how do i do PersonalTeacherPage using React, Next, MUI for my website? Assume that i alrready have all needed components'},
    {"role": 'assistant', 'content': optimized}
]}

samples = []
for i in range(10):
    samples.append(sample)

def dicts_to_jsonl(data_list: list, filename: str) -> None:
    """Save a list of dicts to a jsonl file."""
    with open(filename, "w") as outfile:
        for entry in data_list:
            json.dump(entry, outfile)
            outfile.write("\n")
dicts_to_jsonl(samples, 'sample.jsonl')
dataset_filepath = 'sample.jsonl'

In [13]:
file = client.files.create(
  file=open("sample.jsonl", "rb"),
  purpose="fine-tune"
)
print(file.id)


file-NSX1fdKKMbDXxuGBGDUDrrWo
file-dItB0EeI067892SY1jkCsMi5


In [15]:
suffix_name = "fictadvisor-advisor-proof-of-concept"
job = client.fine_tuning.jobs.create(
  training_file=file.id,
  #validation_file=filev.id, 
  model="gpt-3.5-turbo"
)
print(job.id)
print(job.status)
print(job)

ftjob-omn6mq7HLMo4AfsodeVmm6bI
validating_files
FineTuningJob(id='ftjob-omn6mq7HLMo4AfsodeVmm6bI', created_at=1703167316, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-ptYJ2wDw5nK1CjWbG0joWavk', result_files=[], status='validating_files', trained_tokens=None, training_file='file-dItB0EeI067892SY1jkCsMi5', validation_file=None)


In [20]:
response = openai.fine_tuning.jobs.list_events(job.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)
# here model should be trained

Created fine-tuning job: ftjob-omn6mq7HLMo4AfsodeVmm6bI
Validating training file: file-dItB0EeI067892SY1jkCsMi5
Files validated, moving job to queued state
Fine-tuning job started


In [22]:
# Getting model id on end of training 
response = openai.fine_tuning.jobs.retrieve(job.id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:personal::8YE0sAwS


# Inference 

In [25]:
query = "how do I do PersonalTeacherPage using React, Next, MUI for my website? Assume that I already have all the needed components"
completion = client.chat.completions.create(
    model=fine_tuned_model_id,
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": query}
    ]
)
print(completion.choices[0].message.content)


createContext, Dispatch, FC, SetStateAction, useEffect, useState,} from 'react'; PersonalTeacherPageProps, TeachersPageTabs,} from '@/components/pages/personal-teacher-page/utils';// TODO: move context to separate folder, move types to separate folderexport interface TeacherContext { floatingCardShowed: boolean; setFloatingCardShowed: Dispatch<SetStateAction<boolean>>; teacher: Teacher;}export const teacherContext = createContext<TeacherContext>({ floatingCardShowed: false, setFloatingCardShowed: () => {}, teacher: {} as Teacher,});const PersonalTeacherPage: FC<PersonalTeacherPageProps> = ({ isLoading, isError, data, teacher, query, teacherId,}) => { const router = useRouter(); const { push } = router; const toast = useToast(); const [floatingCardShowed, setFloatingCardShowed] = useState(false); const { tab } = query; const [index, setIndex] = useState<TeachersPageTabs>( TeachersPageTabs.GENERAL, ); const handleChange = useTabState({ tab, router, setIndex }); useEffect(() => { if (isEr

check the cook book please https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb